In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

# Step 1: Loading and sort data. Also ensuring proper date time format
data = pd.read_csv("first_25000_rows.csv")
data['ts_event'] = pd.to_datetime(data['ts_event'])
data = data.sort_values('ts_event')

# Step 2: Calculating price differences at the best levels (0)
data['bid_price_diff'] = data['bid_px_00'] - data['bid_px_00'].shift(1)
data['ask_price_diff'] = data['ask_px_00'] - data['ask_px_00'].shift(1)



# +1 for a bid (buyer pressure) ; -1 for sell (seller pressure)
data['ind_bid'] = np.where(data['bid_price_diff'] > 0, 1,
                   np.where(data['bid_price_diff'] < 0, -1, 0))
data['ind_ask'] = np.where(data['ask_price_diff'] > 0, -1,
                   np.where(data['ask_price_diff'] < 0, 1, 0))

# computing size changes
data['delta_bid_sz'] = data['bid_sz_00'].diff().fillna(0)
data['delta_ask_sz'] = data['ask_sz_00'].diff().fillna(0)

# signed pressure × size change
data['ofi_best'] = data['ind_bid'] * data['delta_bid_sz'] + data['ind_ask'] * data['delta_ask_sz']

# Step 2: Multi-Level OFI
multi_ofi_levels = []
for level in range(10): #(Levels 0–9)
    bid_px_chg = data[f'bid_px_{level:02d}'] - data[f'bid_px_{level:02d}'].shift(1)
    ask_px_chg = data[f'ask_px_{level:02d}'] - data[f'ask_px_{level:02d}'].shift(1)

    # Determining direction of pressure
    ind_bid = np.where(bid_px_chg > 0, 1, np.where(bid_px_chg < 0, -1, 0))
    ind_ask = np.where(ask_px_chg > 0, -1, np.where(ask_px_chg < 0, 1, 0))

    # Compute Size Change
    delta_bid = data[f'bid_sz_{level:02d}'].diff().fillna(0)
    delta_ask = data[f'ask_sz_{level:02d}'].diff().fillna(0)

    # Level OFI = signed size change at N depth
    level_ofi = ind_bid * delta_bid + ind_ask * delta_ask
    multi_ofi_levels.append(level_ofi)

# Sum of all levels
data['ofi_multi'] = np.sum(multi_ofi_levels, axis=0)

# Step 3: Integrated OFI
rolling_window = '600ms'  # Can be '10ms', '1s', '45s', '2min', etc.

data = data.set_index('ts_event')

multi_ofi_matrix = np.vstack(multi_ofi_levels).T

# Perform PCA to extract dominant OFI
pca = PCA(n_components=1)
integrated_ofi = pca.fit_transform(multi_ofi_matrix).flatten()

# Levels the wieghts of the principal component
weights = np.abs(pca.components_[0])
weights /= weights.sum()

# Compute final integrated OFI using weighted combination
data['ofi_integrated'] = multi_ofi_matrix @ weights
data = data.reset_index()

# Step 4: Cross-Asset OFI (Really only works with more than 1 ticker symbol)

cross_asset = data.groupby(['ts_event', 'symbol'])['ofi_best'].sum().reset_index()
# Average across symbols at each timestamp
cross_asset_mean = cross_asset.groupby('ts_event')['ofi_best'].mean().rename("ofi_cross")

# Step 5: Combine Final features
ofi_final = data.set_index('ts_event')[['ofi_best', 'ofi_multi', 'ofi_integrated']]
ofi_final = ofi_final.groupby('ts_event').last()
ofi_final = ofi_final.join(cross_asset_mean, how='left')
ofi_final = ofi_final.fillna(0)

# Step 6: Save and Display
ofi_final.to_csv("ofi_features_custom.csv")
print(f"OFI features saved using {rolling_window} rolling window.")
print(ofi_final)


OFI features saved using 600ms rolling window.
                                     ofi_best  ofi_multi  ofi_integrated  \
ts_event                                                                   
2024-10-21 11:54:29.221064336+00:00       0.0        0.0        0.000000   
2024-10-21 11:54:29.223769812+00:00       0.0        0.0        0.000000   
2024-10-21 11:54:29.225030400+00:00       0.0        0.0        0.000000   
2024-10-21 11:54:29.712434212+00:00       0.0        0.0        0.000000   
2024-10-21 11:54:29.764673165+00:00       0.0        0.0        0.000000   
...                                       ...        ...             ...   
2024-10-21 13:04:16.583527688+00:00       0.0      199.0       33.747655   
2024-10-21 13:04:17.976461017+00:00       0.0        0.0        0.000000   
2024-10-21 13:04:20.085638629+00:00       0.0      199.0       33.747655   
2024-10-21 13:04:20.085651109+00:00       0.0      199.0       45.025130   
2024-10-21 13:04:20.130842270+00:00      